### Imports

In [59]:
%run "./instance.ipynb"

In [9]:
# import numpy as np
# import gurobipy as gp
# from gurobipy import GRB

### Optimization Problems

In [41]:
def solve_master(potential_hubs, capacity_levels, new_hub_capacities, fixed_set_up_costs_with_capacity_level, commodities):

    # model
    master = gp.Model()

    # variables + objective function
    z = master.addVars(potential_hubs, capacity_levels, obj = fixed_set_up_costs_with_capacity_level, vtype = GRB.BINARY, name = "z")
    eta = master.addVar(1, obj = 1, name = 'eta')

    # model sense
    master.ModelSense = GRB.MINIMIZE

    # constraints (do I need Benders constraints at this point?)
    master.addConstr(gp.quicksum(gp.quicksum(new_hub_capacities[i, q] * z[i, q] for i in potential_hubs) for q in capacity_levels) >= gp.quicksum(demand_flattened[k - 1] for k in commodities))
    master.addConstrs(gp.quicksum(z[i, q] for q in capacity_levels) <= 1 for i in potential_hubs)

    # update and solve the model
    master.update()
    master.optimize()

    # optimal solution
    for var in master.getVars():
        print(f"{var.VarName} = {var.x}")

    return None # master_opt_soln, master_opt_val

In [99]:
def solve_dsp(potential_hubs, capacity_levels, new_hub_capacities, fixed_set_up_costs_with_capacity_level, commodities):

    # model
    dsp = gp.Model()

    # variables + objective function
    alpha = dsp.addVars(commodities, obj = 1, name = "alpha")
    u = dsp.addVars(potential_hubs, commodities, obj = z_hat, lb = 0, name = "u") # check the objective function here (how to get z_hat into here?)
    v = dsp.addVars(potential_hubs, obj = new_hub_capacities * z_hat, lb = 0, name = "v") # check the objective function here (how to get z_hat into here?)

    # model sense
    dsp.ModelSense = GRB.MAXIMIZE

    # constraints
    for i in potential_hubs:
        for j in potential_hubs:
            for k in commodities:
                dsp.addConstr(alpha[k] - u[i, k] - u[j, k] - demand_flattened[k - 1] * v[i] <= cost_of_route(i, j, k))
                dsp.addConstr(alpha[k] - u[i, k] - demand_flattened[k - 1] * v[i] <= cost_of_route(i, j, k))

    # update and solve the model
    dsp.update()
    dsp.optimize()

    # optimal solution
    for var in dsp.getVars():
        print(f"{var.VarName} = {var.x}")

    return None # dsp_opt_soln, dsp_opt_val

In [100]:
# model
dsp = gp.Model()

# variables + objective function
alpha = dsp.addVars(commodities, obj = 1, name = "alpha")
u = dsp.addVars(potential_hubs, commodities, obj = 1, lb = 0, name = "u") # check the objective function here
v = dsp.addVars(potential_hubs, obj = 1, lb = 0, name = "v")

# model sense
dsp.ModelSense = GRB.MAXIMIZE

# constraints
for i in potential_hubs:
    for j in potential_hubs:
        for k in commodities:
            dsp.addConstr(alpha[k] - u[i, k] - u[j, k] - demand_flattened[k - 1] * v[i] <= cost_of_route(i, j, k))
            dsp.addConstr(alpha[k] - u[i, k] - demand_flattened[k - 1] * v[i] <= cost_of_route(i, j, k))

# update and solve the model
dsp.update()
dsp.optimize()

# optimal solution
for var in dsp.getVars():
    print(f"{var.VarName} = {var.x}")

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [6]:
def solve_gap(commodities):

    # model
    gap = gp.Model()

    # variables + objective function
    x_hat = gap.addVars(a_i[k], commodities, obj = F, vtype = GRB.BINARY, name = "x_hat")

    # model sense
    gap.ModelSense = GRB.MINIMIZE

    # constraints
    gap.addConstrs(x[a_i[k], k] == 1 for k in commodities)
    gap.addConstrs(gp.quicksum(demand_flattened[k - 1] * x[a_i[k], k] for k in K) <= b[i, q_hat] for i in H(S_prime))

    # update and solve the model
    gap.update()
    gap.optimize()

    # optimal solution
    for var in gap.getVars():
        print(f"{var.VarName} = {var.x}")

    return None # gap_opt_soln, gap_opt_val


In [113]:
def get_a_hat(i, k):

    min_cost = np.inf

    for j in nodes:
        if cost_of_route(i, j, k) < min_cost:
            min_cost = cost_of_route(i, j, k)
    return min_cost

In [114]:
get_a_hat(1, 5)

354145.5120034844

### Algorithms

In [7]:
def benders_decomp():
    upper_bound = 0
    t = 0
    dsp_extreme_pts = set()
    terminate = False

    while terminate == False:
        master_opt_soln, master_opt_val = solver_master() 

        if master_opt_val == upper_bound:
            terminate = True
        else:
            dsp_opt_soln, dsp_opt_val = solve_dual_subproblem()
            dsp_extreme_pts.add(dsp_opt_soln)

            if dsp_opt_val + ... < upper_bound:
                upper_bound = dsp_opt_val + ...

        t += 1

    return None # benders_opt_soln

In [12]:
# def exact_nmchlp():
#     master_opt_soln, master_opt_val = solver_master()
#     gap_opt_soln, gap_opt_val = solver_master()
#     upper_bound = min(min[...], gap_opt_soln, np.inf)

#     for i, q in zip(H(S*), Q_i(S*)):
#         z_iq = 0
#         master_opt_soln, master_opt_val = solver_master()

#         if master_opt_val > upper_bound:
#             z_iq = 1
        
#     for i, q in zip(H*, Q_i*):
#         z_iq = 1
#         master_opt_soln, master_opt_val = solver_master()

#         if master_opt_val > upper_bound:
#             Q_i* = Q_i*.pop(q)

#             if len(Q_i*) == 0:
#                 H* = H*.pop(i)

#     S = set((i, q) for i, q in zip(H*, Q_i*))

#     for S' subset S s.t. (19), (20) hold:
#         # Sovle GAP problem

#         if phi(GAP(z(S'))) < UB:
#             UB = phi(GAP(z(S')))
                           
#     return None